# Import Necessary Modules

In [1]:
import os , time
import numpy as np
import cv2
import glob
import pickle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from random import randint
from IPython import display

from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.cross_validation import train_test_split

print("Imports done")

Imports done


/home/khaled/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Read data

In [2]:
# read image file names from project directory
def readImages_fn(dir, pattern):
    images = []
    for dirpath, dirnames, filenames in os.walk(dir):
        for dirname in dirnames:
            images.append(glob.glob(dir + '/' + dirname + '/' + pattern))
    return images

vehicles_fn = readImages_fn('./training_images/vehicles/vehicles', '*.png')
non_vehicles_fn = readImages_fn('./training_images/non-vehicles/non-vehicles', '*.png')


In [3]:
# Getting no. of elements in the dataset and visualize a random image in each class
cars = []
notcars = []
[cars.extend(vehicles_fn[i]) for i in range(len(vehicles_fn))]
[notcars.extend(non_vehicles_fn[i]) for i in range(len(non_vehicles_fn))]

[None, None]

# Features extraction functions

In [4]:
# # Define a function to compute HOG features 
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

# # Define a function to compute the spacial features 
def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))


# # Define a function to compute color histogram features 
def color_hist(img, nbins=32):    #bins_range=(0, 256)
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Features extraction Main function

In [5]:
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features

# Features extraction paramaters

In [6]:
color_space_param='YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
spatial_size_param=(32, 32) # Spatial binning dimensions
hist_bins_param=32 # 16    # Number of histogram bins
orient_param=9
pix_per_cell_param=8  #HOG
cell_per_block_param=2  # HOG
hog_channel_param= 'ALL' # # Can be 0, 1, 2, or "ALL"

# Training

In [8]:
print ("cars", len(cars))
orientation_range =[8,9]
spatial_size_range=[16,32]
colors_range=['RGB', 'HSV', 'HLS', 'YCrCb'] # LUV, YUV are not tested! An error is occuring for those colors

print ('color_space \torient_param\tspatial_size\tAccuracy')
for orient_param in orientation_range:
    for spatial_size_param_val in spatial_size_range:
        for color_space_param in colors_range:
            spatial_size_param = (spatial_size_param_val,spatial_size_param_val)
            car_features = extract_features(cars, color_space=color_space_param, 
                                    spatial_size=spatial_size_param, hist_bins=hist_bins_param, 
                                    orient=orient_param, pix_per_cell=pix_per_cell_param, 
                                    cell_per_block=cell_per_block_param, 
                                    hog_channel=hog_channel_param)
            notcar_features = extract_features(notcars, color_space=color_space_param, 
                                    spatial_size=spatial_size_param, hist_bins=hist_bins_param, 
                                    orient=orient_param, pix_per_cell=pix_per_cell_param, 
                                    cell_per_block=cell_per_block_param, 
                                    hog_channel=hog_channel_param)

            # Create an array stack of feature vectors
            X = np.vstack((car_features, notcar_features)).astype(np.float64)

            # Define the labels vector
            y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))
            #print ("all data" ,y)
            # Split up data into randomized training and test sets
            rand_state = np.random.randint(0, 100)
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=rand_state)

            #print ("train",y_train)

            # Fit a per-column scaler
            X_scaler = StandardScaler().fit(X_train)
            # Apply the scaler to X
            X_train = X_scaler.transform(X_train)
            X_test = X_scaler.transform(X_test)
            # Use a linear SVC 
            svc = LinearSVC()
            # Check the training time for the SVC
            t=time.time()
            svc.fit(X_train, y_train)
            t2 = time.time()
            print (color_space_param,'\t', orient_param, '\t', spatial_size_param,'\t',round(svc.score(X_test, y_test), 4))

cars 8791
color_space 	orient_param	spatial_size	Accuracy


/home/khaled/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


RGB 	 8 	 (16, 16) 	 0.9823
HSV 	 8 	 (16, 16) 	 0.9913
HLS 	 8 	 (16, 16) 	 0.9916
YCrCb 	 8 	 (16, 16) 	 0.9913
RGB 	 8 	 (32, 32) 	 0.984
HSV 	 8 	 (32, 32) 	 0.993
HLS 	 8 	 (32, 32) 	 0.9907
YCrCb 	 8 	 (32, 32) 	 0.9896
RGB 	 9 	 (16, 16) 	 0.9854
HSV 	 9 	 (16, 16) 	 0.9899
HLS 	 9 	 (16, 16) 	 0.9916
YCrCb 	 9 	 (16, 16) 	 0.9907
RGB 	 9 	 (32, 32) 	 0.9862
HSV 	 9 	 (32, 32) 	 0.9904
HLS 	 9 	 (32, 32) 	 0.9918
YCrCb 	 9 	 (32, 32) 	 0.9918
